In [7]:
import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

import numpy as np
import pandas as pd
import json
import os

In [8]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\AnsatzComp\DW\DW32")
potential = 'DW'
cutoff = 32

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])

    success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [9]:
df = pd.DataFrame({
    'folder': folders,
    #'num_vqe': num_vqe,
    #'shots': shots,
    'tolerance': tolerance,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [10]:
df[['Delta min E', 'Delta median E']] = df[['Delta min E', 'Delta median E']].astype(float)
df = df.sort_values('Delta median E', ascending=False)
df.to_excel(os.path.join(folder_path, "table.xlsx"), index=False)
df

,folder,tolerance,Converged Runs,Mean Iter,Delta min E,Delta median E,Exact,Time
6,RY_CRY_3RY_3CRY_RY_CRY,1.000000e-08,100/100,163,5.15183,2.402370,8.91632e-01,13:11:23.384850
3,RY_CRY_3RY_2CRY,1.000000e-08,100/100,124,6.49666,1.605400,8.91632e-01,9:25:50.774775
4,RY_CRY_3RY_3CRY,1.000000e-08,100/100,138,10.62300,1.392760,8.91632e-01,10:52:34.799230
5,RY_CRY_3RY_3CRY_RY,1.000000e-08,100/100,150,7.83953,1.387620,8.91632e-01,12:06:04.091751
8,RY_CRY_RY,1.000000e-08,100/100,72,7.86402,0.509699,8.91632e-01,4:41:14.683819
7,RY_CRY_3RY_CRY,1.000000e-08,100/100,113,8.82801,0.293858,8.91632e-01,8:28:12.833560
2,RY_CRY_3RY,1.000000e-08,100/100,100,11.03340,0.234497,8.91632e-01,7:05:45.180020
0,RY_CRY,1.000000e-08,100/100,57,9.76666,0.144791,8.91632e-01,3:06:45.405936
1,RY_CRY_2RY,1.000000e-08,100/100,83,7.87674,0.073834,8.91632e-01,5:42:03.224883
